<a href="https://colab.research.google.com/github/aakhterov/ML_projects/blob/master/news_sentiment_analysis/collect_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting a news dataset

The idea is to fine-tune one of the LLM from the Huggingface ecosystem to make a news sentiment analysis regarding the pro- and anti-Israel attitudes (but collected dataset can be used also for other NLP tasks). Here one of the main issue is data labeling. To overcome it we suppose that almost all news from the Palestinian "news" agency "WAFA" and Quatar agency "Al Mayadeen" has anti-Israel position. Most of the Al Jazeera news also tends to be against Israel. In return news by "The Times of Israel" is mostly pro-Israel.
For example, the following piece of news carries an anti-Israel pattern:

> *KHAN YUNIS, Sunday, December 10, 2023 (WAFA) - At least 10 civilians were killed, mostly children, and dozens more were wounded early this morning as Israeli warplanes bombed a residential house in Khan Yunis, south of the Gaza Strip, as the Israeli aggression on the enclave enters its 65th day in a row.
(WAFA "news" agency)*

Opposite the following information from "The Times of Israel" is pro-Israel:

>*Several thousand people demonstrate against antisemitism in Berlin as Germany grapples with a large increase in anti-Jewish incidents following Hamas’s assault on Israel two months ago.
Police estimate that around 3,200 people gathered in the rain in the German capital, while organizers put the figure at 10,000, German news agency dpa reports. Participants in the protest, titled “Never again is now,” march to the Brandenburg Gate.*

We've accumulated news from the following sources:

1. BBC (live news) - from 2023-11-05 to 2023-11-18. Total: 805
2. The Times of Israel (live news) - from 2023-10-07 to 2023-11-18. Total: 6581
3. Al Jazeera (live news) - from 2023-11-04 to 2023-11-25. Total: 3297
4. Al Mayadeen (articles from the site) - from 2023-10-08 to 2023-11-24. Total: 74
5. WAFA "News" Agency (articles from the site section "Occupation")- from 2023-09-28 to 2023-11-26. Total: 1020
6. CNN live news  - from 2023-10-26 to 2023-11-26. Total: 1428

As we aim to use the Huggingface ecosystem and its library for the models fine-tuning we used Dataset object from the Huggingface package here.
The dataset has the following fields:
- "url" - link to the piece of news;
- "datetime" - news date and time (YYYY-mm-ddTHH:MM:SS);
- "author" - news author if provided else None;
- "title" - news title;
- "text" - news text;
- "provider" - news provider
- "source" - where news was collected

All news is collected using Beautiful Soup library. In the case of the BBC site, the WAFA site and the Al Mayadeen site it was enough to use simple GET requests and in the other cases we used Silenium.

In [ ]:
!pip install datasets nltk

In [2]:
import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Colab Notebooks/'

Mounted at /content/drive


## 1. Collecting news from the BBC site

In [54]:
PAUSE_BETWEEN_QUERIES = 1

In [ ]:
current_live_url = 'https://www.bbc.com/news/live/world-middle-east-67446662'

In [ ]:
# We will collect news to the following Python dictionary
data = {"url": [], "datetime": [], "author": [], "title": [], "text": [], "provider": [], "source": []}
current_date = datetime(2023, 11, 18)

# Starting from "current_live_url" grab news from this page and try to find the link to the previous day at the bottom of the page.

while current_live_url and current_date>=datetime(2023, 10, 7):
  print(f"current_date: {current_date.strftime('%Y-%m-%d')}")
  print(f"current_live_url: {current_live_url}, total pages: {total_pages}")

  for page in tqdm(range(1, total_pages+1), 'Pages: '):
    url = f"{current_live_url}/page/{page}"
    result = requests.get(url)
    body = BeautifulSoup(result.text, 'html.parser')

    for article in body.find_all('article'):
      title_block = article.find('span', class_=re.compile('header-text'))
      title = title_block.string if title_block is not None else title_block

      time_block = article.find('time')
      time_block = time_block.find('span', class_=re.compile("qa-post-auto-meta"))
      if time_block is not None:
        datetime_parts = time_block.string.split()
        today = datetime.today()
        if len(datetime_parts) == 1:
          datetime_str = f"{today.year}-{today.month}-{today.day}T{datetime_parts[0]}:00"
          post_datetime = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S')
        else:
          month = datetime.strptime(datetime_parts[2], '%b').month
          day = datetime_parts[1]
          datetime_str = f"{today.year}-{month}-{day}T{datetime_parts[0]}:00"
          post_datetime = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S')

        current_date = post_datetime
      else:
        post_datetime = None

      contributor_block = article.find('div', class_=re.compile('contributor-body'))
      if contributor_block is not None:
        name = contributor_block.find('p', class_=re.compile('contributor-name'))
        name = name.string if name else ''
        role = contributor_block.find('p', class_=re.compile('contributor-role'))
        role = role.string if role else ''
        author = f"{name};{role}"
      else:
        author = None

      text_block = article.find('div', class_=re.compile('post-body'))
      text = []
      if text_block is not None:
        text = [p_block.text if p_block is not None else '' for p_block in text_block.find_all('p')]

      text = '\n'.join(text)

      data['url'].append(url)
      data['title'].append(title)
      data['datetime'].append(post_datetime.strftime('%Y-%m-%dT%H:%M:%S') if post_datetime else None)
      data['author'].append(author if author else '-')
      data['text'].append(text)
      data['provider'].append('BBC')
      data['source'].append('site-live-news')

    time.sleep(PAUSE_BETWEEN_QUERIES)

  current_live_url = article.find('a', href=re.compile('https://www.bbc.com/news/live/world-middle-east'))
  if current_live_url is None:
    current_live_url = article.find('a', href=re.compile('https://www.bbc.co.uk/news/live/world-middle-east'))

  if current_live_url is not None:
    current_live_url = current_live_url.get('href')
    result = requests.get(current_live_url)
    body = BeautifulSoup(result.text, 'html.parser')
    total_pages = int(body.find('span', class_=re.compile('qa-pagination-total-page-number')).string)

In [56]:
# Instantiate Dataset object from the dictionary
ds_bbc = Dataset.from_dict(data)
ds_bbc

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 805
})

In [ ]:
ds_bbc.save_to_disk(base_path + '/Data/bbc_news_ds')

In [ ]:
# ds_bbc = load_from_disk(base_path + '/Data/bbc_news_ds')

## 2. Collecting live news from the "THE TIMES OF ISRAEL" site

To collect news from the "THE TIMES OF ISRAEL" site we used the Selenium library. As it's a bit difficult to use selenium inside Google Colab, we leveraged a simple Python script that can be found in [Github](https://github.com/aakhterov/ML_projects/blob/master/news_sentiment_analysis/main_toi.py). Here we just load the saved dataset.

In [5]:
ds_toi = load_from_disk(base_path + '/Data/toi_news_ds')
ds_toi

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 6581
})

In [ ]:
ds_toi[:3]

{'url': ['https://www.timesofisrael.com/liveblog_entry/us-official-says-there-will-only-be-a-significant-pause-in-gaza-when-hostages-released/',
  'https://www.timesofisrael.com/liveblog_entry/survey-shows-substantial-support-for-renewal-of-jewish-settlement-in-gaza-after-war/',
  'https://www.timesofisrael.com/liveblog_entry/hamas-claims-that-gaza-death-toll-reaches-12300/'],
 'datetime': ['2023-11-18T19:30:14',
  '2023-11-18T19:25:21',
  '2023-11-18T19:18:28'],
 'author': ['AFP', '-', 'AFP'],
 'title': ['US official says there will only be a ‘significant pause’ in Gaza when hostages released',
  'Survey shows substantial support for renewal of Jewish settlement in Gaza after war',
  'Hamas claims that Gaza death toll reaches 12,300'],
 'text': ['US President Joe Biden’s main adviser on the Middle East says there will only be a “significant pause” in the Israel-Hamas war if hostages held by Hamas and other terror groups in Gaza are freed.\n“The surge in humanitarian relief, the surge 

In [6]:
def change_provider_and_source(example):
  example['provider'] = 'The Times of Israel'
  example['source'] = 'site-live-news'
  return example

In [7]:
ds_toi = ds_toi.map(change_provider_and_source)

## 3. Collect live news from Al Jazeera site

Here is the same case as with "THE TIMES OF ISRAEL" - we need to use Selenium to grab the news, save the collected dataset (code is [here](https://github.com/aakhterov/ML_projects/blob/master/news_sentiment_analysis/main_aj.py)) and load it here.

In [8]:
ds_aj = load_from_disk(base_path + '/Data/aljazeera_news_ds')
ds_aj

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 3317
})

In [9]:
ds_aj = ds_aj.filter(lambda x: "This live page is now closed" not in x["text"])

In [10]:
def change_provider_and_source(example):
  example['provider'] = 'Al Jazeera'
  example['source'] = 'site-live-news'
  return example

In [11]:
ds_aj = ds_aj.map(change_provider_and_source)

## 4. Collect articles from Al Mayadeen site

In [ ]:
BASE_URL = "https://english.almayadeen.net"
FIRST_URL = "https://english.almayadeen.net/coverage/operation-al-aqsa-flood"
PAGE_URL = "https://english.almayadeen.net/GetMore/coverage/operation-al-aqsa-flood?widgetid=00000000-0000-0000-0000-000000000000&widget=coverage-articles&postid=1753428&ismain=False&order=1"

In [ ]:
data = {"url": [], "datetime": [], "author": [], "title": [], "text": [], "provider": [], "source": []}

res = requests.get(FIRST_URL)
first_page_body = BeautifulSoup(res.text, 'html.parser')
first_block = first_page_body.find('div', attrs={"data-widgetkey": "coverage-articles"})

# Collect all news link
urls = []
for div in first_block.find_all('div', class_='content-block'):
    urls.append(div.find('a').get('href'))

for page in range(1, 101):
    print(f"Page {page}. Length urls: {len(urls)}")
    res = requests.get(f"{PAGE_URL}&page={page}")
    page_body = BeautifulSoup(res.text, 'html.parser')
    divs = page_body.find_all('div', class_='content-block')
    if divs:
        for div in divs:
            urls.append(div.find('a').get('href'))
    else:
        break

print(f"Total amount of the news: {len(urls)}")

# Grab the news using the collected links
for idx, url in enumerate(urls):
    print(idx+1, url)
    result = requests.get(BASE_URL + url)
    body = BeautifulSoup(result.text, 'html.parser')
    article_block = body.find('article', class_ = 'post-details')
    if article_block:
        title = article_block.find('h1').text.strip()

        author_block = article_block.find('a', class_='post-author post-author-with-img')
        author = author_block.text.strip() if author_block else None

        datetime_block = article_block.find('li', class_='media-date')
        if datetime_block:
            datetime_ = datetime_block.text.strip()
            if datetime_.split()[0] == 'Today':
                post_datetime = datetime.strptime(datetime.today().strftime("%d %b %Y") + ' ' + datetime_.split()[1], "%d %b %Y %H:%M") # 15 Oct 2023 06:36
            else:
                post_datetime = datetime.strptime(datetime_, "%d %b %Y %H:%M") # 15 Oct 2023 06:36
        else:
            post_datetime = None

        text_block = body.find('div', class_='p-content')
        text_lst = []
        if text_block:
            for p in text_block.find_all('p'):
                text_lst.append(p.text)
        text = '\n'.join(text_lst)

        print('\t', post_datetime, title)
        data['url'].append(BASE_URL + url)
        data['title'].append(title)
        data['datetime'].append(post_datetime.strftime('%Y-%m-%dT%H:%M:%S') if post_datetime else None)
        data['author'].append(author if author else '-')
        data['text'].append(text)
        data['provider'].append('Al Mayadeen')
        data['source'].append('site-articles')

In [ ]:
ds = Dataset.from_dict(data)
ds.save_to_disk('al_mayadeen_news_ds')

In [ ]:
# ds_am = load_from_disk(base_path + '/Data/al_mayadeen_news_ds')
# ds_am

## 5. Collect news from WAFA "News" Agency site

In [ ]:
BASE_URL = "https://english.wafa.ps"
NEWS_URL = BASE_URL + "/Regions/Details/2"

In [ ]:
data = {"url": [], "datetime": [], "author": [], "title": [], "text": [], "provider": [], "source": []}

for page in range(51):
    print(f"Page: {page}")
    result = requests.get(NEWS_URL, params={'pageNumber': page})
    body = BeautifulSoup(result.text, "html.parser")
    posts_block = body.find('div', class_="post-blockcat-wrapper")
    for div in posts_block.find_all('div', class_="content"):
        datetime_block = div.find('span', class_="meta-item date")
        post_datetime = datetime.strptime(datetime_block.text, "%d/%B/%Y %I:%M %p") # <span class="meta-item date">07/October/2023 02:29 PM</span>

        url_block = div.find('h4', class_="title")
        post_url = BASE_URL + url_block.find('a').get("href")

        print('\t', post_datetime, post_url)
        res = requests.get(post_url)
        html = BeautifulSoup(res.text, "html.parser")
        post_block = html.find('div', class_="blog-wrap")

        title_block = post_block.find('h3', class_='title')
        title = title_block.text if title_block else None

        text_lst = []
        for p in post_block.find('div', class_='content').find_all('p'):
            if p is not None:
                text_lst.append(p.text)
        text = '\n'.join(text_lst)

        author = text_lst[-1]

        data['url'].append(post_url)
        data['title'].append(title)
        data['datetime'].append(post_datetime.strftime('%Y-%m-%dT%H:%M:%S') if post_datetime else None)
        data['author'].append(author if author else '-')
        data['text'].append(text)
        data['provider'].append('WAFA News Agency')
        data['source'].append('site-occupation')

In [ ]:
ds_wafa = Dataset.from_dict(data)
ds_wafa.save_to_disk('wafa_news_ds')
ds_wafa

In [ ]:
# ds_wafa = load_from_disk(base_path + '/Data/wafa_news_ds')
# ds_wafa

## 6. Collect live news from CNN's site


Here is the same case as with "THE TIMES OF ISRAEL" - we need to use Selenium to grab the news, save the collected dataset (code is [here](https://github.com/aakhterov/ML_projects/blob/master/news_sentiment_analysis/main_cnn.py)) and load it here.

In [18]:
ds_cnn = load_from_disk(base_path + '/Data/cnn_news_ds')
ds_cnn

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 1428
})

## 6. Merging the collected datasets

In [20]:
ds = concatenate_datasets([ds_bbc, ds_toi, ds_aj, ds_am, ds_wafa, ds_cnn])

In [ ]:
ds

Dataset({
    features: ['url', 'datetime', 'author', 'title', 'text', 'provider', 'source'],
    num_rows: 13395
})

In [ ]:
ds.unique('provider')

['BBC',
 'The Times of Israel',
 'Al Jazeera',
 'Al Mayadeen',
 'WAFA News Agency',
 'CNN']

In [ ]:
ds.save_to_disk(base_path + '/Data/news_ds')

Saving the dataset (0/1 shards):   0%|          | 0/13395 [00:00<?, ? examples/s]

## 7. Calculating the average news length

In [ ]:
def get_length(example):
  example['len'] = len(example['text'].split())
  return example

In [ ]:
ds_w_len = ds.map(get_length)

Map:   0%|          | 0/13395 [00:00<?, ? examples/s]

In [ ]:
ds_w_len.set_format("pandas")
ds_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])

<ipython-input-65-08d70b6d2378>:2: FutureWarning: ['url', 'datetime', 'author', 'title', 'text'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  ds_w_len[:].groupby(["provider", "source"]).agg(['mean', 'count'])


len      
                                                                   mean count
provider            source                                                   
Al Jazeera          site-live-news                           101.171975  3297
Al Mayadeen         site-articles                           1196.256757    74
BBC                 site-live-news                           129.027295   806
                    site-tag(News)-search(israel)            569.427350   117
                    site-tag(News)-search(israel-gaza war)   537.027778    72
CNN                 site-live-news                           197.663165  1428
The Times of Israel site-live-news                           127.901383  6581
WAFA News Agency    site-occupation                          158.311765  1020